**Importing the needed libraries.**













In [2]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Image Data Generation/Augmentation**

In [5]:
import cv2,os
data_path = "/content/drive/MyDrive/dataset"
categories=os.listdir(data_path)
labels = [i for i in range(len(categories))]
label_dict= dict(zip(categories,labels))
print(label_dict)
print(categories)
print(labels)

{'with_mask': 0, 'without_mask': 1}
['with_mask', 'without_mask']
[0, 1]


In [ ]:
import cv2
img_size = 100
data= []
target = []
for category in categories:
  folder_path = os.path.join(data_path, category)
  img_names = os.listdir(folder_path)
  
  for img_name in img_names: 
    img_path = os.path.join(folder_path, img_name)
    img = cv2.imread(img_path)
    try: 
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      resized = cv2.resize(gray, (img_size, img_size))
      data.append(resized)
      target.append(label_dict[category])
    except Exception as e:
      print('Exception : ', e)

In [ ]:
import numpy as np
data = np.array(data) /255.0 
data = np.reshape(data, (data.shape[0], img_size, img_size, 1))
target = np.array(target)
from keras.utils import np_utils 
new_target = np_utils.to_categorical(target)

In [ ]:
np.save('data', data)
np.save('target', new_target)

**Build the neural network**

In [3]:
model = Sequential([
    Conv2D(200, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

KeyboardInterrupt: ignored

**Saving best model**

In [ ]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

**Train the model**

In [ ]:
history = model.fit(train_generator,
                              epochs=20,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/20
131/131 [==============================] - 14s 104ms/step - loss: 0.0864 - acc: 0.9655 - val_loss: 0.1200 - val_acc: 0.9536
Epoch 2/20
131/131 [==============================] - 13s 102ms/step - loss: 0.0634 - acc: 0.9786 - val_loss: 0.0248 - val_acc: 0.9948
Epoch 3/20
131/131 [==============================] - 13s 102ms/step - loss: 0.0990 - acc: 0.9655 - val_loss: 0.0602 - val_acc: 0.9794
Epoch 4/20
131/131 [==============================] - 14s 103ms/step - loss: 0.0713 - acc: 0.9717 - val_loss: 0.0487 - val_acc: 0.9794
Epoch 5/20
131/131 [==============================] - 14s 106ms/step - loss: 0.0754 - acc: 0.9724 - val_loss: 0.0420 - val_acc: 0.9794
Epoch 6/20
131/131 [==============================] - 14s 108ms/step - loss: 0.0739 - acc: 0.9740 - val_loss: 0.0257 - val_acc: 0.9897
Epoch 7/20
131/131 [==============================] - 14s 105ms/step - loss: 0.0652 - acc: 0.9732 - val_loss: 0.0235 - val_acc: 0.9897
Epoch 8/20
131/131 [==============================] - 1

**Download the best Model checkpoint**

In [ ]:
!zip -r /content/BestModel.zip /content/model2-015.model

  adding: content/model2-015.model/ (stored 0%)
  adding: content/model2-015.model/variables/ (stored 0%)
  adding: content/model2-015.model/variables/variables.index (deflated 64%)
  adding: content/model2-015.model/variables/variables.data-00000-of-00001 (deflated 39%)
  adding: content/model2-015.model/saved_model.pb (deflated 89%)
  adding: content/model2-015.model/assets/ (stored 0%)


In [ ]:
from google.colab import files
files.download('BestModel.zip') 

**TEST**

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("/content/model2-015.model")